### osu!nn #1: Map Dataset Reader

This notebook reads a file "maplist.txt", then reads the .osu files and the relevant music files to convert into some data.

Data that feeds the Deep Neural Network.

Last edit: 2019/4/22

First of all, we need to install FFmpeg and specify its path here. It is needed to convert the .mp3 files to .wavs which Python can read.

It's also fine to use any other converter, such as LAME: just edit the 24th line of osureader.py (starting with "subprocess.call") for the converter's parameters.

**Then, fill maplist.txt  with the paths of .osu files you want to train with.** Otherwise it cannot find any of the maps because the maps are on my computer. The default model is trained with the Sota dataset including 44 maps of Sota Fujimori music.

After that run the grid below to convert the maps.

In [ ]:
import os, re, time
from osureader import * 

# set the ffmpeg path here!!
# add "r" before the path string

GLOBAL_VARS["ffmpeg_path"] = r"D:\StudyData\Tensorflow\ffmpeg\bin\ffmpeg.exe";

# in linux, it is installed globally, so use this
# GLOBAL_VARS["ffmpeg_path"] = "ffmpeg";

mapdata_path = "mapdata/";

# check if it works
test_process_path(GLOBAL_VARS["ffmpeg_path"]);

# check if nodejs works
test_process_path("node");

# the divisor parameter
divisor = 4;

# make sure the mapdata folder exists
if not os.path.isdir(mapdata_path):
    os.mkdir(mapdata_path);

with open("maplist.txt") as fp:
    fcont = fp.readlines();

# The following part is something I used to filter maps with difficulty names
results = [];
# exclude_words = ["Easy", "Normal", "Hard", "Taiko", "Salad", "Platter", "Overdose", "Rain", "4K", "5K", "6K", "7K", "8K", "9K",
#                  "Kantan", "Futsuu", "Muzukashii", "Oni", "Field "];
for line in fcont:
#     if re.search("TV", line):
#         apd = True;
#         for kw in exclude_words:
#             if kw.lower() in line.strip().lower():
#                 apd = False;
#                 break;
#         if apd:
#             results.append(line.strip());
    results.append(line);

# Remove the originally existing npzs
for file in os.listdir(mapdata_path):
    if file.endswith(".npz"):
        os.remove(os.path.join(mapdata_path, file));

print("Number of filtered maps: {}".format(len(results)));

for k, mname in enumerate(results):
    try:
        start = time.time()
        read_and_save_osu_file(mname.strip(), filename=os.path.join(mapdata_path, str(k)), divisor=divisor);
        end = time.time()
        print("Map data #" + str(k) + " saved! time = " + str(end - start) + " secs");
    except Exception as e:
        print("Error on #{}, path = {}, error = {}".format(str(k), mname.strip(), e));

# If some map causes bug please tell me!! https://discord.gg/npmSy7K